In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from time import sleep
from random import randint
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
secrets_file = open("SpotifySecret.txt","r")

In [3]:
string = secrets_file.read()

In [4]:
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        secrets_dict[line.split(':')[0]]=line.split(':')[1]

In [5]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['cid'],
                                                           client_secret=secrets_dict['cs']))

In [7]:
# loading list of songs collected using scraping
scraping_list = pd.read_csv('scraping_list.csv')

In [8]:
scraping_list.head()

,artist,song
0,Pink Floyd,Another Brick in the Wall (part 2)
1,Barbra Streisand,A Woman in Love
2,John Lennon,(Just Like) Starting Over
3,Diana Ross,Upside Down
4,Lipps Inc,Funkytown


In [9]:
# loading list of songs collected from Spotify using API
spotify_list = pd.read_csv('spotify_playlist.csv')

In [12]:
spotify_list.head()

,artist,song
0,Prince,When Doves Cry
1,Irene Cara,"Fame - From ""Fame"" Original Motion Picture Sou..."
2,Bananarama,Venus
3,a-ha,Take On Me
4,Madonna,Material Girl


In [11]:
# removing 'track_id' column
spotify_list=spotify_list.drop(['Unnamed: 0', 'track_id'], axis=1)

In [14]:
repository=pd.concat([scraping_list, spotify_list], axis=0)

In [15]:
repository

,artist,song
0,Pink Floyd,Another Brick in the Wall (part 2)
1,Barbra Streisand,A Woman in Love
2,John Lennon,(Just Like) Starting Over
3,Diana Ross,Upside Down
4,Lipps Inc,Funkytown
...,...,...
125,Bonnie Tyler,Total Eclipse of the Heart
126,Madonna,Like a Prayer
127,Donna Summer,She Works Hard For The Money
128,Bonnie Tyler,Total Eclipse of the Heart


In [16]:
repository['artist']=repository['artist'].apply(lambda x: x.lower())
repository['song']=repository['song'].apply(lambda x: x.lower())
repository=repository.drop_duplicates()
repository=repository.reset_index()

In [17]:
repository=repository.drop(['index'], axis=1)

In [18]:
repository.head()

,artist,song
0,pink floyd,another brick in the wall (part 2)
1,barbra streisand,a woman in love
2,john lennon,(just like) starting over
3,diana ross,upside down
4,lipps inc,funkytown


In [19]:
repository.shape

(1810, 2)

In [27]:
# collecting audio features from Spotify
songs_uri=[]
songs_danceability=[]
songs_energy=[]
songs_key=[]
songs_loudness=[]
songs_mode=[]
songs_speechiness=[]
songs_acousticness=[]
songs_instrumentalness=[]
songs_liveness=[]
songs_valence=[]
songs_tempo=[]
songs_duration_ms=[]
songs_time_signature=[]

for i in range(len(repository)):
    results = sp.search(q=(repository.loc[i,'artist']+ ' ' +repository.loc[i,'song']), limit=1)
#    for j in range(len(results['tracks']['items'])):
#        results_artist=results['tracks']['items'][j]['artists'][0]['name']
#        results_song=results['tracks']['items'][j]['name']
#        print(repository2.loc[4,'artist'])
#         print(repository2.loc[i,'song'])
#        if ((repository.loc[12,'song']==results_song) & (repository.loc[12,'artist']==results_artist)):
# #            uri_id=results['tracks']['items'][0]['uri']
    if len(results['tracks']['items'])>0:
        songs_uri.append(results['tracks']['items'][0]['uri'])
        uri=results['tracks']['items'][0]['uri']
        results=sp.audio_features(uri)    
        songs_danceability.append(results[0]['danceability'])
        songs_energy.append(results[0]['energy'])
        songs_key.append(results[0]['key'])
        songs_loudness.append(results[0]['loudness'])
        songs_mode.append(results[0]['mode'])
        songs_speechiness.append(results[0]['speechiness'])
        songs_acousticness.append(results[0]['acousticness'])
        songs_instrumentalness.append(results[0]['instrumentalness'])
        songs_liveness.append(results[0]['liveness'])
        songs_valence.append(results[0]['valence'])
        songs_tempo.append(results[0]['tempo'])
        songs_duration_ms.append(results[0]['duration_ms'])
        songs_time_signature.append(results[0]['time_signature'])
    else:
        songs_uri.append('NA')
        songs_danceability.append('NA')
        songs_energy.append('NA')
        songs_key.append('NA')
        songs_loudness.append('NA')
        songs_mode.append('NA')
        songs_speechiness.append('NA')
        songs_acousticness.append('NA')
        songs_instrumentalness.append('NA')
        songs_liveness.append('NA')
        songs_valence.append('NA')
        songs_tempo.append('NA')
        songs_duration_ms.append('NA')
        songs_time_signature.append('NA')
    wait_time = randint(1,2)
 #   print("I will sleep for " + str(wait_time) + " second/s.")
    sleep(wait_time)

KeyboardInterrupt: 

In [21]:
# adding audio features to my list of songs
repository['uri']=songs_uri
repository['danceability']=songs_danceability
repository['energy']=songs_energy
repository['key']=songs_key
repository['loudness']=songs_loudness
repository['mode']=songs_mode
repository['speechiness']=songs_speechiness
repository['acousticness']=songs_acousticness
repository['instrumentalness']=songs_instrumentalness
repository['liveness']=songs_liveness
repository['valence']=songs_valence
repository['tempo']=songs_tempo
repository['duration_ms']=songs_duration_ms
repository['time_signature']=songs_time_signature

In [22]:
repository.head()

,artist,song,uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,pink floyd,another brick in the wall (part 2),spotify:track:1xUTI8Ce8IQnyYCGPmrbzS,0.395,0.766,0,-11.041,1,0.0512,0.00583,0.00624,0.774,0.611,105.41,326373,4
1,barbra streisand,a woman in love,spotify:track:53No4na49hTw82MxrFh2PB,0.489,0.245,3,-16.876,0,0.0284,0.43,0.000015,0.298,0.333,169.747,230227,4
2,john lennon,(just like) starting over,spotify:track:5y0YreEOnQiKFAnCrcFIXz,0.701,0.79,9,-6.16,1,0.0725,0.301,0.000061,0.179,0.421,99.104,236547,4
3,diana ross,upside down,spotify:track:3pbtBomO4Zt5gGiqsYeiBH,0.873,0.855,5,-8.559,0,0.0615,0.179,0.0285,0.0377,0.884,107.868,245600,4
4,lipps inc,funkytown,spotify:track:5dIKF6uVJ206hj0pO0gg0I,0.893,0.509,0,-10.136,1,0.0659,0.00154,0.515,0.0783,0.334,122.478,469347,4


In [23]:
repository.shape

(1810, 16)

In [24]:
# some of the queries returned no results ('NA', Not Available in audio features rows)
repository['time_signature'].value_counts()

4     1727
NA      54
3       28
1        1
Name: time_signature, dtype: int64

In [25]:
# . I'm going to remove this rows with no audio features
repository=repository[~(repository['time_signature']=='NA')]

In [26]:
repository.to_csv('list_with_audio_features.csv')